In [80]:
#Jupyterlab Ctrl + /  一键注释代码区域

In [81]:
#include packages

In [2]:
import backtrader as bt
from datetime import datetime
import pandas as pd
import pf_api
import matplotlib.pyplot as plt
import tushare as ts
pro = ts.pro_api()
import numpy as np

In [3]:
#use pf_api
# df_msft = pf_api.query_stocks(codes='MSFT', start='2015-01-03', end='2015-06-30')['MSFT']
# df_msft

In [4]:
#use tushare

In [5]:
ts.set_token('11c9226df8d261027dfdaf9efa556e0fe51efa01420ec9067fa18207')

In [78]:
# pre_aapl=pro.us_daily(ts_code='AAPL', start_date='20190101', end_date='20190904')#调用美股：https://tushare.pro/document/2?doc_id=254
# dataframe = ts.pro_bar(ts_code='600000.SH', adj='qfq', start_date='20190101',end_date='20200710')
dataframe = ts.pro_bar(ts_code='300215.SZ', adj='qfq', start_date='20230201',end_date='20240201')#301156.SZ，301258.SZ，300215.SZ

In [79]:
dataframe #Raw Data

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,300215.SZ,20240201,4.15,4.15,3.96,4.04,4.16,-0.12,-2.8846,76264.65,30987.614
1,300215.SZ,20240131,4.31,4.39,4.10,4.16,4.42,-0.26,-5.8824,95627.30,40399.071
2,300215.SZ,20240130,4.54,4.57,4.39,4.42,4.47,-0.05,-1.1186,72532.00,32538.234
3,300215.SZ,20240129,4.64,4.68,4.47,4.47,4.62,-0.15,-3.2468,42990.28,19533.792
4,300215.SZ,20240126,4.61,4.71,4.60,4.62,4.62,0.00,0.0000,49826.00,23198.285
...,...,...,...,...,...,...,...,...,...,...,...
244,300215.SZ,20230207,5.85,5.92,5.84,5.92,5.85,0.07,1.1966,57079.01,33597.741
245,300215.SZ,20230206,5.79,5.90,5.78,5.85,5.82,0.03,0.5155,51594.69,30162.076
246,300215.SZ,20230203,5.79,5.84,5.72,5.82,5.80,0.02,0.3448,49271.37,28531.902
247,300215.SZ,20230202,5.83,5.85,5.79,5.80,5.84,-0.04,-0.6849,49547.04,28835.522


In [80]:
#Data Converting

In [81]:
dataframe.sort_index(inplace=True, ascending=False)

In [82]:
print(dataframe)

       ts_code trade_date  open  high   low  close  pre_close  change  \
248  300215.SZ   20230201  5.69  5.85  5.66   5.84       5.69    0.15   
247  300215.SZ   20230202  5.83  5.85  5.79   5.80       5.84   -0.04   
246  300215.SZ   20230203  5.79  5.84  5.72   5.82       5.80    0.02   
245  300215.SZ   20230206  5.79  5.90  5.78   5.85       5.82    0.03   
244  300215.SZ   20230207  5.85  5.92  5.84   5.92       5.85    0.07   
..         ...        ...   ...   ...   ...    ...        ...     ...   
4    300215.SZ   20240126  4.61  4.71  4.60   4.62       4.62    0.00   
3    300215.SZ   20240129  4.64  4.68  4.47   4.47       4.62   -0.15   
2    300215.SZ   20240130  4.54  4.57  4.39   4.42       4.47   -0.05   
1    300215.SZ   20240131  4.31  4.39  4.10   4.16       4.42   -0.26   
0    300215.SZ   20240201  4.15  4.15  3.96   4.04       4.16   -0.12   

     pct_chg       vol     amount  
248   2.6362  68764.00  39661.476  
247  -0.6849  49547.04  28835.522  
246   0.3448  4

In [83]:
dataframe['trade_date'] = pd.to_datetime(dataframe['trade_date'])
print(dataframe.head())

       ts_code trade_date  open  high   low  close  pre_close  change  \
248  300215.SZ 2023-02-01  5.69  5.85  5.66   5.84       5.69    0.15   
247  300215.SZ 2023-02-02  5.83  5.85  5.79   5.80       5.84   -0.04   
246  300215.SZ 2023-02-03  5.79  5.84  5.72   5.82       5.80    0.02   
245  300215.SZ 2023-02-06  5.79  5.90  5.78   5.85       5.82    0.03   
244  300215.SZ 2023-02-07  5.85  5.92  5.84   5.92       5.85    0.07   

     pct_chg       vol     amount  
248   2.6362  68764.00  39661.476  
247  -0.6849  49547.04  28835.522  
246   0.3448  49271.37  28531.902  
245   0.5155  51594.69  30162.076  
244   1.1966  57079.01  33597.741  


In [84]:
#Convert into Panda Dataframes
data = bt.feeds.PandasData(
    dataname=dataframe,
    datetime='trade_date',  # If the date column is not an index, specify the column number or name; 
                            #set to None if it's an index column
    open=2,  # Column containing the opening prices
    high=3,  # Column containing the highest prices
    low=4,  # Column containing the lowest prices
    close=5,  # Column containing the closing prices
    volume=9,  # Column containing the trading volume
    openinterest=-1,  # No column for open interest (openinterest is used in futures trading)
    fromdate=datetime(2023, 2, 1),  # Start date
    todate=datetime(2024, 2, 1)  # End date
)

In [85]:
#Quantitative Grid Trading
#1. Fix grid A 以10为单位从0划分网格（价格向下取整x10）
#2. Fix grid B 以close price +- 网格size划分网格
#3. Percentage grid 以close price的百分制划分网格

In [86]:
print(11.3//0.5)#整除

22.0


# 策略A：等距固定网格

In [87]:
#df = pd.DataFrame()

In [222]:
class QuantitativeGridTradingStrategy_FixGrid_A(bt.Strategy):

    params = (
        ('buy_multiplier',1),
        # ('buy_share',1000),
        ('sell_multiplier',1),
        # ('sell_share',3000),
        ('window',12),
        ('grid_multiplier',3),
        ('printlog', True),
        
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        # dt = dt or self.datas[0].datetime.date(0)
        # print('%s, %s' % (dt.isoformat(), txt))
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # self.grid_size = 0.03 #网格间距【范围：(0,1)】
        self.grid_size = 0.01*self.params.grid_multiplier #网格间距【范围：(0,1)】
        self.init_long = 10000 #初始买入底仓
        
        self.yesterday_price = self.data.close[0] #上一天的价格
        self.buy_share = 1000*self.params.buy_multiplier #注：这里当前预设的是达成条件后买卖指定的share，也可以改成达成条件买卖指定金额
        # self.sell_share = 3000
        self.sell_share = 1000*self.params.sell_multiplier
        # self.window = 20 # rolling window of Bollinger Bands

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, {order.executed.price}, {order.executed.size} shares')
                
            elif order.issell():
                self.log(f'Sell EXECUTED, {order.executed.price}, {order.executed.size} shares')
                

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
            

        # Write down: no pending order
        self.order = None
    
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        
        
        
    def next(self):
        UpperBB=0
        LowerBB=0
        print(f'Close price today: {self.data.close[0]}')
        
        if len(self)>self.params.window: #Calculate Bollinger Bands
            #Calculate rolling mean and std
            sumOfRoll=0
            #Mean
            for i in np.arange(0,self.params.window):#0 1 2 3 4      #roll back (windowsize) days
                sumOfRoll+=self.data.close[-1*i]#0 -1 -2 -3 -4
                # print(i)
            rollingMean=sumOfRoll/self.params.window
            # print(f'Rolling Mean: {rollingMean}')

            #Std
            sumOfSquare=0
            for i in np.arange(0,self.params.window):
                sumOfSquare+=(self.data.close[-1*i]-rollingMean)**2
            rollingVar=sumOfSquare/self.params.window
            rollingStd=np.sqrt(rollingVar)
            # print(f'Rolling Std: {rollingStd}')
            UpperBB = rollingMean + (2 * rollingStd)
            LowerBB = rollingMean - (2 * rollingStd)
            print(f'UpperBB: {UpperBB}')
            print(f'LowerBB: {LowerBB}')

        
        close_price=self.data.close[0] #获取当天的closing price
        #print(close_price)
        
        if (len(self)==1):
            self.buy(size=self.init_long) #买入初始持仓

        # else: #测试：读取上一天和当天的close是不是正确
        #     print(f'today price is: {close_price}, yesterday: {self.yesterday_price}')

        #----------------------------当grid size>1时用-------------------------------------------
        # #按照grid size为倍数取整划分网格，10倍向下取整，例如54变50：floor(54/10)*10=50，向上取十：ceil(54/10)*10=60
        # elif (close_price>=np.ceil(self.yesterday_price/self.grid_size)*self.grid_size):
        #     #如果当天收盘价上穿前一天价格所处网格的顶部，则卖出预设的Shares/金额
        #     self.sell(size=self.sell_share)


        # elif (close_price<np.floor(self.yesterday_price/self.grid_size)*self.grid_size):
        #     #如果当天收盘价下穿前一天价格所处网格的底部，则买入预设的Shares/金额
        #     self.buy(size=self.buy_share)
        #-----------------------------------------------------------------------------------------


        #----------------------------当0<grid size<1时用-------------------------------------------
        #例如网格大小是0.5时，网格下限：11.3//0.5*0.5，上限：(11.3//0.5+1)*0.5
        elif (close_price>=(self.yesterday_price//self.grid_size+1)*self.grid_size):
            #如果当天收盘价上穿前一天价格所处网格的顶部，则卖出预设的Shares/金额
            if len(self)<=self.params.window:
                self.sell(size=self.sell_share)
            elif close_price<=UpperBB and len(self)>self.params.window:#但是如果价格高过布林带（上限），也就是没有达到这个条件，那么就不卖了
                self.sell(size=self.sell_share)
            elif close_price>UpperBB and len(self)>self.params.window:
                print('Upcrossing Bolinger Band, do not sell today.')
                # pass

        elif (close_price<(self.yesterday_price//self.grid_size)*self.grid_size):
            #如果当天收盘价下穿前一天价格所处网格的底部，则买入预设的Shares/金额
            if len(self)<=self.params.window:
                self.buy(size=self.buy_share)
            elif close_price>=LowerBB and len(self)>self.params.window:#但是如果价格低于布林带（下限），那么就不买了
                self.buy(size=self.buy_share)
            elif close_price<LowerBB and len(self)>self.params.window:
                print('Downcrossing Bolinger Band, do not buy today.')
                # pass
        #-----------------------------------------------------------------------------------------
        

        #else: #检测无操作日用
            #print('do nothing') 
            #print(position.size)
        
        self.yesterday_price=close_price #记录当天收盘价传递给下一天
        
#    def stop(self):
        # self.log(
        #     f'(Channel Periods: {self.p.H_period},{self.p.L_period}) - \
        # Final Portfolio Value: {self.broker.getvalue():.2f}',
        #     doprint=True)
        #print(len(self))

    # def stop(self):
    #     # self.log('(MA Period %2d) Ending Value %.2f' %
    #     #         (self.params.window, self.broker.getvalue()), doprint=True)
    #     # self.log('(Buy Multiplier %2d) Ending Value %.2f' %
    #     #         (self.params.buy_multiplier, self.broker.getvalue()), doprint=True)
    #     # self.log('(Sell Multiplier %2d) Ending Value %.2f' %
    #     #         (self.params.sell_multiplier, self.broker.getvalue()), doprint=True)
    #     self.log('(Grid Multiplier %2d) Ending Value %.2f' %
    #             (self.params.grid_multiplier, self.broker.getvalue()), doprint=True)

In [223]:
if __name__ == '__main__':
    %matplotlib inline
    cerebro = bt.Cerebro()
    # pfdata = pf_api.PFData(codes='MSFT', start='2019-01-01', end='2020-01-01')['MSFT']
    init_cash = 1000000.0#100000.0
    cerebro.broker.setcash(init_cash)
    cerebro.addstrategy(QuantitativeGridTradingStrategy_FixGrid_A) #A策略

    cerebro.adddata(data)

    # strats = cerebro.optstrategy(QuantitativeGridTradingStrategy_FixGrid_A, window=range(3, 25))
    # strats = cerebro.optstrategy(QuantitativeGridTradingStrategy_FixGrid_A, buy_multiplier=range(1, 5))
    # strats = cerebro.optstrategy(QuantitativeGridTradingStrategy_FixGrid_A, sell_multiplier=range(1, 5))
    # strats = cerebro.optstrategy(QuantitativeGridTradingStrategy_FixGrid_A, grid_multiplier=range(1, 15))
    
    cerebro.run(maxcpus=1)
    value = cerebro.broker.get_value()
    cash_balance = cerebro.broker.get_cash()
    position = cerebro.broker.getposition(data)
    ret = (value-init_cash)/init_cash
    print (f'The return of the strategy is: {ret}')
    print (f'The cash left is: {cash_balance}')
    print (f'Current position: {position.size} shares of MSFT')
    cerebro.plot(iplot=False)


2024-02-01, (Grid Multiplier  3) Ending Value 1006120.00
2024-02-01, (Grid Multiplier  1) Ending Value 1006120.00
2024-02-01, (Grid Multiplier  3) Ending Value 998420.00
2024-02-01, (Grid Multiplier  2) Ending Value 998420.00
2024-02-01, (Grid Multiplier  3) Ending Value 989880.00
2024-02-01, (Grid Multiplier  3) Ending Value 989880.00
2024-02-01, (Grid Multiplier  3) Ending Value 1001470.00
2024-02-01, (Grid Multiplier  4) Ending Value 1001470.00
2024-02-01, (Grid Multiplier  3) Ending Value 994490.00
2024-02-01, (Grid Multiplier  5) Ending Value 994490.00
2024-02-01, (Grid Multiplier  3) Ending Value 981880.00
2024-02-01, (Grid Multiplier  6) Ending Value 981880.00
2024-02-01, (Grid Multiplier  3) Ending Value 987360.00
2024-02-01, (Grid Multiplier  7) Ending Value 987360.00
2024-02-01, (Grid Multiplier  3) Ending Value 989670.00
2024-02-01, (Grid Multiplier  8) Ending Value 989670.00
2024-02-01, (Grid Multiplier  3) Ending Value 1001400.00
2024-02-01, (Grid Multiplier  9) Ending Val

# B：当天收盘价上下浮动指定范围视为网格

In [224]:
class QuantitativeGridTradingStrategy_FixGrid_B(bt.Strategy):

    params = (
        ('grid_multiplier',15),
        ('printlog', True),
        
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        # dt = dt or self.datas[0].datetime.date(0)
        # print('%s, %s' % (dt.isoformat(), txt))
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # self.grid_size = 0.015 #网格间距，实际网格大小是间距*2
        self.grid_size = 0.001*self.params.grid_multiplier #网格间距，实际网格大小是间距*2
        self.init_long = 10000 #初始买入底仓
        self.yesterday_price = self.data.close[0] #上一天的价格
        self.buy_share = 1000 #注：这里当前预设的是达成条件后买卖指定的share，也可以改成达成条件买卖指定金额
        self.sell_share = 3000
        self.window = 20 # rolling window of Bollinger Bands

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, {order.executed.price}, {order.executed.size} shares')
            elif order.issell():
                self.log(f'Sell EXECUTED, {order.executed.price}, {order.executed.size} shares')

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None
    
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        
        
    def next(self):

        UpperBB=0
        LowerBB=0
        print(f'Close price today: {self.data.close[0]}')
        if len(self)>self.window: #Calculate Bollinger Bands
            #Calculate rolling mean and std
            sumOfRoll=0
            #Mean
            for i in np.arange(0,self.window):#0 1 2 3 4      #roll back (windowsize) days
                sumOfRoll+=self.data.close[-1*i]#0 -1 -2 -3 -4
                # print(i)
            rollingMean=sumOfRoll/self.window
            # print(f'Rolling Mean: {rollingMean}')

            #Std
            sumOfSquare=0
            for i in np.arange(0,self.window):
                sumOfSquare+=(self.data.close[-1*i]-rollingMean)**2
            rollingVar=sumOfSquare/self.window
            rollingStd=np.sqrt(rollingVar)
            # print(f'Rolling Std: {rollingStd}')
            UpperBB = rollingMean + (2 * rollingStd)
            LowerBB = rollingMean - (2 * rollingStd)
            print(f'UpperBB: {UpperBB}')
            print(f'LowerBB: {LowerBB}')


        
        close_price=self.data.close[0] #获取当天的closing price
        #print(close_price)
        
        if (len(self)==1):
            self.buy(size=self.init_long) #买入初始持仓

        # else: #测试：读取上一天和当天的close是不是正确
        #     print(f'today price is: {close_price}, yesterday: {self.yesterday_price}')


        #前一天价格 +- grid size
        elif (close_price>=self.yesterday_price+self.grid_size):
            #如果当天收盘价上穿前一天价格所处网格的顶部，则卖出预设的Shares/金额
            # self.sell(size=self.sell_share)
            if len(self)<=self.window:
                self.sell(size=self.sell_share)
            elif close_price<=UpperBB and len(self)>self.window:#但是如果价格高过布林带（上限），那么就不卖了
                self.sell(size=self.sell_share)
            elif close_price>UpperBB and len(self)>self.window:
                print('Upcrossing Bolinger Band, do not sell today.')
                # pass


        elif (close_price<self.yesterday_price-self.grid_size):
            #如果当天收盘价下穿前一天价格所处网格的底部，则买入预设的Shares/金额
            # self.buy(size=self.buy_share)
            if len(self)<=self.window:
                self.buy(size=self.buy_share)
            elif close_price>=LowerBB and len(self)>self.window:#但是如果价格低于布林带（下限），那么就不买了
                self.buy(size=self.buy_share)
            elif close_price<LowerBB and len(self)>self.window:
                print('Downcrossing Bolinger Band, do not buy today.')
                # pass

        # else: #检测用
        #     print('do nothing') 
            #print(position.size)
        
        self.yesterday_price=close_price #记录当天收盘价传递给下一天
        
    # def stop(self):
    #     # self.log('(MA Period %2d) Ending Value %.2f' %
    #     #         (self.params.window, self.broker.getvalue()), doprint=True)
    #     # self.log('(Buy Multiplier %2d) Ending Value %.2f' %
    #     #         (self.params.buy_multiplier, self.broker.getvalue()), doprint=True)
    #     # self.log('(Sell Multiplier %2d) Ending Value %.2f' %
    #     #         (self.params.sell_multiplier, self.broker.getvalue()), doprint=True)
    #     self.log('(Grid Multiplier %2d) Ending Value %.2f' %
    #             (self.params.grid_multiplier, self.broker.getvalue()), doprint=True)

In [230]:
if __name__ == '__main__':
    %matplotlib inline
    cerebro = bt.Cerebro()
    # pfdata = pf_api.PFData(codes='MSFT', start='2019-01-01', end='2020-01-01')['MSFT']
    init_cash = 1000000.0#100000.0
    cerebro.broker.setcash(init_cash)
    cerebro.addstrategy(QuantitativeGridTradingStrategy_FixGrid_B) #A策略

    cerebro.adddata(data)

    # strats = cerebro.optstrategy(QuantitativeGridTradingStrategy_FixGrid_A, window=range(3, 25))
    # strats = cerebro.optstrategy(QuantitativeGridTradingStrategy_FixGrid_A, buy_multiplier=range(1, 5))
    # strats = cerebro.optstrategy(QuantitativeGridTradingStrategy_FixGrid_A, sell_multiplier=range(1, 5))
    # strats = cerebro.optstrategy(QuantitativeGridTradingStrategy_FixGrid_B, grid_multiplier=range(1, 50))
    
    cerebro.run(maxcpus=1)
    value = cerebro.broker.get_value()
    cash_balance = cerebro.broker.get_cash()
    position = cerebro.broker.getposition(data)
    ret = (value-init_cash)/init_cash
    print (f'The return of the strategy is: {ret}')
    print (f'The cash left is: {cash_balance}')
    print (f'Current position: {position.size} shares of MSFT')
    cerebro.plot(iplot=False)


2024-02-01, (Grid Multiplier 15) Ending Value 1428370.00
2024-02-01, (Grid Multiplier  1) Ending Value 1428370.00
2024-02-01, (Grid Multiplier 15) Ending Value 1428370.00
2024-02-01, (Grid Multiplier  2) Ending Value 1428370.00
2024-02-01, (Grid Multiplier 15) Ending Value 1428370.00
2024-02-01, (Grid Multiplier  3) Ending Value 1428370.00
2024-02-01, (Grid Multiplier 15) Ending Value 1428370.00
2024-02-01, (Grid Multiplier  4) Ending Value 1428370.00
2024-02-01, (Grid Multiplier 15) Ending Value 1428370.00
2024-02-01, (Grid Multiplier  5) Ending Value 1428370.00
2024-02-01, (Grid Multiplier 15) Ending Value 1428370.00
2024-02-01, (Grid Multiplier  6) Ending Value 1428370.00
2024-02-01, (Grid Multiplier 15) Ending Value 1428370.00
2024-02-01, (Grid Multiplier  7) Ending Value 1428370.00
2024-02-01, (Grid Multiplier 15) Ending Value 1428370.00
2024-02-01, (Grid Multiplier  8) Ending Value 1428370.00
2024-02-01, (Grid Multiplier 15) Ending Value 1428370.00
2024-02-01, (Grid Multiplier  9

# C：当天收盘价上下浮动指定百分比视为网格

In [231]:
class QuantitativeGridTradingStrategy_Dynm_Grid_C(bt.Strategy):

    params = (
        ('grid_multiplier',3),
        ('printlog', True),
        
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        # dt = dt or self.datas[0].datetime.date(0)
        # print('%s, %s' % (dt.isoformat(), txt))
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # self.grid_size = 0.003 #网格间距，百分制，实际网格大小是间距*2
        self.grid_size = 0.0001*self.params.grid_multiplier #网格间距，百分制，实际网格大小是间距*2
        self.init_long = 10000 #初始买入底仓
        self.yesterday_price = self.data.close[0] #上一天的价格
        self.buy_share = 1000 #注：这里当前预设的是达成条件后买卖指定的share，也可以改成达成条件买卖指定金额
        self.sell_share = 3000
        self.window = 20 # rolling window of Bollinger Bands

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, {order.executed.price}, {order.executed.size} shares')
            elif order.issell():
                self.log(f'Sell EXECUTED, {order.executed.price}, {order.executed.size} shares')

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None
    
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        
        
    def next(self):

        UpperBB=0
        LowerBB=0
        print(f'Close price today: {self.data.close[0]}')
        if len(self)>self.window: #Calculate Bollinger Bands
            #Calculate rolling mean and std
            sumOfRoll=0
            #Mean
            for i in np.arange(0,self.window):#0 1 2 3 4      #roll back (windowsize) days
                sumOfRoll+=self.data.close[-1*i]#0 -1 -2 -3 -4
                # print(i)
            rollingMean=sumOfRoll/self.window
            # print(f'Rolling Mean: {rollingMean}')

            #Std
            sumOfSquare=0
            for i in np.arange(0,self.window):
                sumOfSquare+=(self.data.close[-1*i]-rollingMean)**2
            rollingVar=sumOfSquare/self.window
            rollingStd=np.sqrt(rollingVar)
            # print(f'Rolling Std: {rollingStd}')
            UpperBB = rollingMean + (2 * rollingStd)
            LowerBB = rollingMean - (2 * rollingStd)
            print(f'UpperBB: {UpperBB}')
            print(f'LowerBB: {LowerBB}')



        
        close_price=self.data.close[0] #获取当天的closing price
        #print(close_price)
        
        if (len(self)==1):
            self.buy(size=self.init_long) #买入初始持仓

        # else: #测试：读取上一天和当天的close是不是正确
        #     print(f'today price is: {close_price}, yesterday: {self.yesterday_price}')


        #前一天价格 +- grid size
        elif (close_price>=self.yesterday_price*(1+self.grid_size)):
            #如果当天收盘价上穿前一天价格所处网格的顶部，则卖出预设的Shares/金额
            # self.sell(size=self.sell_share)
            if len(self)<=self.window:
                self.sell(size=self.sell_share)
            elif close_price<=UpperBB and len(self)>self.window:#但是如果价格高过布林带（上限），那么就不卖了
                self.sell(size=self.sell_share)
            elif close_price>UpperBB and len(self)>self.window:
                print('Upcrossing Bolinger Band, do not sell today.')
                # pass
        


        elif (close_price<self.yesterday_price*(1-self.grid_size)):
            #如果当天收盘价下穿前一天价格所处网格的底部，则买入预设的Shares/金额
            # self.buy(size=self.buy_share)
            if len(self)<=self.window:
                self.buy(size=self.buy_share)

            elif close_price>=LowerBB and len(self)>self.window:#但是如果价格低于布林带（下限），那么就不买了
                self.buy(size=self.buy_share)
            elif close_price<LowerBB and len(self)>self.window:
                print('Downcrossing Bolinger Band, do not buy today.')
                # pass


        # else: #检测用
        #     print('do nothing') 
        #     #print(position.size)
        
        self.yesterday_price=close_price #记录当天收盘价传递给下一天

    def stop(self):
        # self.log('(MA Period %2d) Ending Value %.2f' %
        #         (self.params.window, self.broker.getvalue()), doprint=True)
        # self.log('(Buy Multiplier %2d) Ending Value %.2f' %
        #         (self.params.buy_multiplier, self.broker.getvalue()), doprint=True)
        # self.log('(Sell Multiplier %2d) Ending Value %.2f' %
        #         (self.params.sell_multiplier, self.broker.getvalue()), doprint=True)
        # self.log('(Grid Multiplier %2d) Ending Value %.2f' %
        #         (self.params.grid_multiplier, self.broker.getvalue()), doprint=True)

In [232]:
if __name__ == '__main__':
    %matplotlib inline
    cerebro = bt.Cerebro()
    # pfdata = pf_api.PFData(codes='MSFT', start='2019-01-01', end='2020-01-01')['MSFT']
    init_cash = 1000000.0#100000.0
    cerebro.broker.setcash(init_cash)
    cerebro.addstrategy(QuantitativeGridTradingStrategy_Dynm_Grid_C) #A策略

    cerebro.adddata(data)

    # strats = cerebro.optstrategy(QuantitativeGridTradingStrategy_FixGrid_A, window=range(3, 25))
    # strats = cerebro.optstrategy(QuantitativeGridTradingStrategy_FixGrid_A, buy_multiplier=range(1, 5))
    # strats = cerebro.optstrategy(QuantitativeGridTradingStrategy_FixGrid_A, sell_multiplier=range(1, 5))
    # strats = cerebro.optstrategy(QuantitativeGridTradingStrategy_Dynm_Grid_C, grid_multiplier=range(1, 50))
    
    cerebro.run(maxcpus=1)
    value = cerebro.broker.get_value()
    cash_balance = cerebro.broker.get_cash()
    position = cerebro.broker.getposition(data)
    ret = (value-init_cash)/init_cash
    print (f'The return of the strategy is: {ret}')
    print (f'The cash left is: {cash_balance}')
    print (f'Current position: {position.size} shares of MSFT')
    cerebro.plot(iplot=False)


2024-02-01, (Grid Multiplier  3) Ending Value 1463860.00
2024-02-01, (Grid Multiplier  1) Ending Value 1463860.00
2024-02-01, (Grid Multiplier  3) Ending Value 1463860.00
2024-02-01, (Grid Multiplier  2) Ending Value 1463860.00
2024-02-01, (Grid Multiplier  3) Ending Value 1463860.00
2024-02-01, (Grid Multiplier  3) Ending Value 1463860.00
2024-02-01, (Grid Multiplier  3) Ending Value 1463860.00
2024-02-01, (Grid Multiplier  4) Ending Value 1463860.00
2024-02-01, (Grid Multiplier  3) Ending Value 1463860.00
2024-02-01, (Grid Multiplier  5) Ending Value 1463860.00
2024-02-01, (Grid Multiplier  3) Ending Value 1463860.00
2024-02-01, (Grid Multiplier  6) Ending Value 1463860.00
2024-02-01, (Grid Multiplier  3) Ending Value 1463860.00
2024-02-01, (Grid Multiplier  7) Ending Value 1463860.00
2024-02-01, (Grid Multiplier  3) Ending Value 1463860.00
2024-02-01, (Grid Multiplier  8) Ending Value 1463860.00
2024-02-01, (Grid Multiplier  3) Ending Value 1463860.00
2024-02-01, (Grid Multiplier  9

# D：正金字塔策略（基于等距网格即A策略）

In [235]:
class Positive_Pyramid_Strategy_D(bt.Strategy):

    params = (
        ('grid_multiplier',3),
        ('printlog', True),
        
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        # dt = dt or self.datas[0].datetime.date(0)
        # print('%s, %s' % (dt.isoformat(), txt))
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        self.grid_size = 0.001*self.params.grid_multiplier #网格间距
        self.init_long = 10000 #初始买入底仓
        self.yesterday_price = self.data.close[0] #上一天的价格
        self.buy_share = 1000 #注：每下降一个grid，默认买量+100
        self.sell_share = 3000 #注：每上升一个grid，默认卖量-100
        self.window = 12 # rolling window of Bollinger Bands
        
        # self.order_stack['Shares']=0 #每一天买卖操作的量
        # self.order_stack['Action']=0 #每一天的买/卖状态，0为无
        #self.location=0

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, {order.executed.price}, {order.executed.size} shares')
            elif order.issell():
                self.log(f'Sell EXECUTED, {order.executed.price}, {order.executed.size} shares')

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None
    
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        
        
    def next(self):

        UpperBB=0
        LowerBB=0
        print(f'Close price today: {self.data.close[0]}')
        if len(self)>self.window: #Calculate Bollinger Bands
            #Calculate rolling mean and std
            sumOfRoll=0
            #Mean
            for i in np.arange(0,self.window):#0 1 2 3 4      #roll back (windowsize) days
                sumOfRoll+=self.data.close[-1*i]#0 -1 -2 -3 -4
                # print(i)
            rollingMean=sumOfRoll/self.window
            # print(f'Rolling Mean: {rollingMean}')

            #Std
            sumOfSquare=0
            for i in np.arange(0,self.window):
                sumOfSquare+=(self.data.close[-1*i]-rollingMean)**2
            rollingVar=sumOfSquare/self.window
            rollingStd=np.sqrt(rollingVar)
            # print(f'Rolling Std: {rollingStd}')
            UpperBB = rollingMean + (2 * rollingStd)
            LowerBB = rollingMean - (2 * rollingStd)
            print(f'UpperBB: {UpperBB}')
            print(f'LowerBB: {LowerBB}')


        
        close_price=self.data.close[0] #获取当天的closing price
        #print(close_price)
        
        if (len(self)==1):
            self.buy(size=self.init_long) #买入初始持仓
            # self.order_stack[self.position,'Shares']=self.init_long
        # else: #测试：读取上一天和当天的close是不是正确
        #     print(f'today price is: {close_price}, yesterday: {self.yesterday_price}')


        #例如网格大小是0.5时，网格下限：11.3//0.5*0.5，上限：(11.3//0.5+1)*0.5
        elif (close_price>=(self.yesterday_price//self.grid_size+1)*self.grid_size): #如果连续卖出(触碰到网格顶部并且前一天是卖出)，则按照默认值递减卖出；#如果买转卖，则按照前一天买的量卖出
            # self.sell(size=self.sell_share)
            # self.buy_share=self.sell_share #如果下次转买入了，则买相同的量
            # self.sell_share=np.maximum(self.sell_share-1000,1000) #如果下次还是卖出，则少卖100share
            if len(self)<=self.window:
                self.sell(size=self.sell_share)
                self.buy_share=self.sell_share #如果下次转买入了，则买相同的量
                self.sell_share=np.maximum(self.sell_share-1000,1000) #如果下次还是卖出，则少卖100share

            elif close_price<=UpperBB and len(self)>self.window:#但是如果价格高过布林带（上限），那么就不卖了
                self.sell(size=self.sell_share)
                self.buy_share=self.sell_share #如果下次转买入了，则买相同的量
                self.sell_share=np.maximum(self.sell_share-1000,1000) #如果下次还是卖出，则少卖100share

            elif close_price>UpperBB and len(self)>self.window:
                print('Upcrossing Bolinger Band, do not sell today.')
                # pass
            
        elif (close_price<self.yesterday_price//self.grid_size*self.grid_size):#如果卖转买，按照前一天卖的量买入；如果连续买入，则按照默认值递增买入
            # self.buy(size=self.buy_share)#按照前一天卖出的量买入
            # self.sell_share=self.buy_share#如果下次卖则卖相同量
            # self.buy_share=self.buy_share+1000#如果下次还是买则多买100
            if len(self)<=self.window:
                self.buy(size=self.buy_share)#按照前一天卖出的量买入
                self.sell_share=self.buy_share#如果下次卖则卖相同量
                self.buy_share=self.buy_share+1000#如果下次还是买则多买100
            
            elif close_price>=LowerBB and len(self)>self.window:#但是如果价格低于布林带（下限），那么就不买了
                self.buy(size=self.buy_share)#按照前一天卖出的量买入
                self.sell_share=self.buy_share#如果下次卖则卖相同量
                self.buy_share=self.buy_share+1000#如果下次还是买则多买100
            elif close_price<LowerBB and len(self)>self.window:
                print('Downcrossing Bolinger Band, do not buy today.')
                # pass

        # else: #检测用
        #     print('do nothing') 
        #     #print(position.size)
        
        self.yesterday_price=close_price #记录当天收盘价传递给下一天
        #self.location=self.location+1

    def stop(self):
        # self.log('(MA Period %2d) Ending Value %.2f' %
        #         (self.params.window, self.broker.getvalue()), doprint=True)
        # self.log('(Buy Multiplier %2d) Ending Value %.2f' %
        #         (self.params.buy_multiplier, self.broker.getvalue()), doprint=True)
        # self.log('(Sell Multiplier %2d) Ending Value %.2f' %
        #         (self.params.sell_multiplier, self.broker.getvalue()), doprint=True)
        # self.log('(Grid Multiplier %2d) Ending Value %.2f' %
        #         (self.params.grid_multiplier, self.broker.getvalue()), doprint=True)

In [236]:
if __name__ == '__main__':
    %matplotlib inline
    cerebro = bt.Cerebro()
    # pfdata = pf_api.PFData(codes='MSFT', start='2019-01-01', end='2020-01-01')['MSFT']
    init_cash = 1000000.0#100000.0
    cerebro.broker.setcash(init_cash)
    cerebro.addstrategy(Positive_Pyramid_Strategy_D) #A策略

    cerebro.adddata(data)

    # strats = cerebro.optstrategy(QuantitativeGridTradingStrategy_FixGrid_A, window=range(3, 25))
    # strats = cerebro.optstrategy(QuantitativeGridTradingStrategy_FixGrid_A, buy_multiplier=range(1, 5))
    # strats = cerebro.optstrategy(QuantitativeGridTradingStrategy_FixGrid_A, sell_multiplier=range(1, 5))
    # strats = cerebro.optstrategy(Positive_Pyramid_Strategy_D, grid_multiplier=range(1, 50))
    
    cerebro.run(maxcpus=1)
    value = cerebro.broker.get_value()
    cash_balance = cerebro.broker.get_cash()
    position = cerebro.broker.getposition(data)
    ret = (value-init_cash)/init_cash
    print (f'The return of the strategy is: {ret}')
    print (f'The cash left is: {cash_balance}')
    print (f'Current position: {position.size} shares of MSFT')
    cerebro.plot(iplot=False)


2024-02-01, (Grid Multiplier  3) Ending Value 1020950.00
2024-02-01, (Grid Multiplier  1) Ending Value 1020950.00
2024-02-01, (Grid Multiplier  3) Ending Value 1012220.00
2024-02-01, (Grid Multiplier  2) Ending Value 1012220.00
2024-02-01, (Grid Multiplier  3) Ending Value 1007320.00
2024-02-01, (Grid Multiplier  3) Ending Value 1007320.00
2024-02-01, (Grid Multiplier  3) Ending Value 1005210.00
2024-02-01, (Grid Multiplier  4) Ending Value 1005210.00
2024-02-01, (Grid Multiplier  3) Ending Value 998580.00
2024-02-01, (Grid Multiplier  5) Ending Value 998580.00
2024-02-01, (Grid Multiplier  3) Ending Value 989030.00
2024-02-01, (Grid Multiplier  6) Ending Value 989030.00
2024-02-01, (Grid Multiplier  3) Ending Value 999710.00
2024-02-01, (Grid Multiplier  7) Ending Value 999710.00
2024-02-01, (Grid Multiplier  3) Ending Value 1002320.00
2024-02-01, (Grid Multiplier  8) Ending Value 1002320.00
2024-02-01, (Grid Multiplier  3) Ending Value 1017330.00
2024-02-01, (Grid Multiplier  9) Endi

In [94]:
np.maximum(5,10)#不用np.max

10

# E：倒金字塔策略（基于等距网格即A策略）

In [241]:
class Negative_Pyramid_Strategy_E(bt.Strategy):

    params = (
        ('grid_multiplier',3),
        ('printlog', True),
        
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        # dt = dt or self.datas[0].datetime.date(0)
        # print('%s, %s' % (dt.isoformat(), txt))
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # self.grid_size = 0.013 #网格间距
        self.grid_size = 0.001*self.params.grid_multiplier #网格间距
        self.init_long = 10000 #初始买入底仓
        self.yesterday_price = self.data.close[0] #上一天的价格
        

        self.shares=0
        self.action=0
        self.location=0
        self.sell_to_buy=0

        self.window = 12 # rolling window of Bollinger Bands

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, {order.executed.price}, {order.executed.size} shares')
            elif order.issell():
                self.log(f'Sell EXECUTED, {order.executed.price}, {order.executed.size} shares')

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None
    
    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))
        
        
    def next(self):


        UpperBB=self.data.close[0]
        LowerBB=self.data.close[0]
        print(f'Close price today: {self.data.close[0]}')
        if len(self)>self.window: #Calculate Bollinger Bands
            #Calculate rolling mean and std
            sumOfRoll=0
            #Mean
            for i in np.arange(0,self.window):#0 1 2 3 4      #roll back (windowsize) days
                sumOfRoll+=self.data.close[-1*i]#0 -1 -2 -3 -4
                # print(i)
            rollingMean=sumOfRoll/self.window
            # print(f'Rolling Mean: {rollingMean}')

            #Std
            sumOfSquare=0
            for i in np.arange(0,self.window):
                sumOfSquare+=(self.data.close[-1*i]-rollingMean)**2
            rollingVar=sumOfSquare/self.window
            rollingStd=np.sqrt(rollingVar)
            # print(f'Rolling Std: {rollingStd}')
            UpperBB = rollingMean + (2 * rollingStd)
            LowerBB = rollingMean - (2 * rollingStd)
            print(f'UpperBB: {UpperBB}')
            print(f'LowerBB: {LowerBB}')


        
        close_price=self.data.close[0] #获取当天的closing price
        #print(self.location-1)
        if (len(self)==1):
            self.buy(size=self.init_long) #买入初始持仓
            #self.shares=np.append(self.shares,self.init_long) #记录初始持仓买入
            self.shares=np.append(self.shares,2000) #记录买入量默认值
            self.action=np.append(self.action,'Buy')
        
        #网格下限：11.3//0.5*0.5，上限：(11.3//0.5+1)*0.5
        elif close_price<=UpperBB and close_price>=LowerBB:
            if (close_price>=(self.yesterday_price//self.grid_size+1)*self.grid_size) and (self.action[self.location-1]=='Sell'): #连续卖出，则按照前一天卖的量+100卖出

                
            
                self.sell(size=self.shares[self.location-1]+1000)
                # self.action[self.location]='Sell' #记录卖出
                self.action=np.append(self.action,'Sell') #记录卖出
                self.shares=np.append(self.shares,self.shares[self.location-1]+1000) #记录量


            

            elif (close_price>=(self.yesterday_price//self.grid_size+1)*self.grid_size) and (self.action[self.location-1]=='Buy'): #买转卖，则按照前一天买的量卖出

                self.sell(size=self.sell_to_buy) #按照上一次卖转买的量卖出
                self.action=np.append(self.action,'Sell') #记录卖出
                self.shares=np.append(self.shares,self.sell_to_buy) #记录量

            elif (close_price<self.yesterday_price//self.grid_size*self.grid_size) and (self.action[self.location-1]=='Buy'):#连续买入，按照前一天买的量+100买入

                self.buy(size=self.shares[self.location-1]+1000)
                self.action=np.append(self.action,'Buy') #记录买入
                self.shares=np.append(self.shares,self.shares[self.location-1]+1000) #记录量
    
            elif (close_price<self.yesterday_price//self.grid_size*self.grid_size) and (self.action[self.location-1]=='Sell'):#卖转买，按照前一天卖的量买入
            
                self.buy(size=self.shares[self.location-1])#按照前一天卖出的量买入
                self.action=np.append(self.action,'Buy') #记录买入
                self.shares=np.append(self.shares,self.shares[self.location-1]) #记录量

            
                #-------------------------
                self.sell_to_buy=self.shares[self.location-1]#记录下一次买转卖时的卖量
                #-------------------------

            else:
                self.action=np.append(self.action,self.action[self.location-1]) #无操作则继承前一天的记录
                self.shares=np.append(self.shares,self.shares[self.location-1]) #无操作则继承前一天的记录
        else:
            self.action=np.append(self.action,self.action[self.location-1]) #无操作则继承前一天的记录
            self.shares=np.append(self.shares,self.shares[self.location-1]) #无操作则继承前一天的记录
            print('Upper bound or lower bound exceeded, Buying or Selling aborted!')
                #print(position.size)
        
        self.yesterday_price=close_price #记录当天收盘价传递给下一天
        self.location=self.location+1

    def stop(self):
        # self.log('(MA Period %2d) Ending Value %.2f' %
        #         (self.params.window, self.broker.getvalue()), doprint=True)
        # self.log('(Buy Multiplier %2d) Ending Value %.2f' %
        #         (self.params.buy_multiplier, self.broker.getvalue()), doprint=True)
        # self.log('(Sell Multiplier %2d) Ending Value %.2f' %
        #         (self.params.sell_multiplier, self.broker.getvalue()), doprint=True)
        # self.log('(Grid Multiplier %2d) Ending Value %.2f' %
        #         (self.params.grid_multiplier, self.broker.getvalue()), doprint=True)

In [244]:
if __name__ == '__main__':
    %matplotlib inline
    cerebro = bt.Cerebro()
    # pfdata = pf_api.PFData(codes='MSFT', start='2019-01-01', end='2020-01-01')['MSFT']
    init_cash = 1000000.0#100000.0
    cerebro.broker.setcash(init_cash)
    cerebro.addstrategy(Negative_Pyramid_Strategy_E) #A策略
    # cerebro.addstrategy(QuantitativeGridTradingStrategy_FixGrid_B) #B策略
    # cerebro.addstrategy(QuantitativeGridTradingStrategy_Dynm_Grid_C) #C策略
    # cerebro.addstrategy(Positive_Pyramid_Strategy_D) #D策略
    # cerebro.addstrategy(Negative_Pyramid_Strategy_E) #E策略
    cerebro.adddata(data)

    # strats = cerebro.optstrategy(Negative_Pyramid_Strategy_E, grid_multiplier=range(1, 50))


    
    cerebro.run(maxcpus=1)
    value = cerebro.broker.get_value()
    cash_balance = cerebro.broker.get_cash()
    position = cerebro.broker.getposition(data)
    ret = (value-init_cash)/init_cash
    print (f'The return of the strategy is: {ret}')
    print (f'The cash left is: {cash_balance}')
    print (f'Current position: {position.size} shares of MSFT')
    cerebro.plot(iplot=False)


2024-02-01, (Grid Multiplier  3) Ending Value 934460.00
2024-02-01, (Grid Multiplier  1) Ending Value 934460.00
2024-02-01, (Grid Multiplier  3) Ending Value 934460.00
2024-02-01, (Grid Multiplier  2) Ending Value 934460.00
2024-02-01, (Grid Multiplier  3) Ending Value 873100.00
2024-02-01, (Grid Multiplier  3) Ending Value 873100.00
2024-02-01, (Grid Multiplier  3) Ending Value 873100.00
2024-02-01, (Grid Multiplier  4) Ending Value 873100.00
2024-02-01, (Grid Multiplier  3) Ending Value 934460.00
2024-02-01, (Grid Multiplier  5) Ending Value 934460.00
2024-02-01, (Grid Multiplier  3) Ending Value 873100.00
2024-02-01, (Grid Multiplier  6) Ending Value 873100.00
2024-02-01, (Grid Multiplier  3) Ending Value 882920.00
2024-02-01, (Grid Multiplier  7) Ending Value 882920.00
2024-02-01, (Grid Multiplier  3) Ending Value 873100.00
2024-02-01, (Grid Multiplier  8) Ending Value 873100.00
2024-02-01, (Grid Multiplier  3) Ending Value 873100.00
2024-02-01, (Grid Multiplier  9) Ending Value 87

In [122]:
a=np.zeros(2)

a=np.append(a,[10])#append是返回值，不是对原数组直接操作。
a

array([ 0.,  0., 10.])